In [6]:
import pandas as pd

test_pd = pd.read_csv("SuperCenterDataNew.csv",header=None)
test_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,39,48.0,130.0,147.0,475.0,2998.0,4792.0,5478.0,7146.0,7160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,48,79.0,232.0,340.0,396.0,649.0,911.0,1016.0,1020.0,1172.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32,48.0,65.0,130.0,772.0,816.0,1344.0,3220.0,4445.0,6536.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39,175.0,421.0,806.0,1025.0,1629.0,2424.0,3151.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
